# Output relations

output relation from 2016 to 2022

- relations/y2016_layer5_to_y2022_layer4.csv
- relations/y2016_layer5_to_y2022_tables.csv
- relations/y2016_layer5_to_y2022_others.csv

In [ ]:
"""
output relations/y2016_layer5_to_y2022_layer4.csv
output relations/y2016_layer5_to_y2022_tables.csv
"""

import pandas as pd
import glob
import re
import csv
import os
from os import path
from lib.vars import dirs
from lib.utils import get_glob_file,save_csv
from lib.tables_utils import iter_tables_for_outcome_raw

os.makedirs(f"{dirs().output.relations_2016}", exist_ok=True)

def expand_y2016_id(df:pd.DataFrame)->pd.DataFrame:
    df = df.dropna(subset=["y2016_id"])
    splitted_y2016_ids = df.loc[:,"y2016_id"].str.split(" *,|、 *",regex=True)   
    df.loc[:,"y2016_id"] = splitted_y2016_ids
    df = df.explode("y2016_id")
    df.loc[:,"y2016_id"] = df.loc[:,"y2016_id"].str.replace("(なし)|(新設)|(新規)| |　","",regex=True)
    return df[df["y2016_id"]!=""]    


# output y2016_layer5_to_y2022_layer4.csv
df = pd.DataFrame([])
for filename in glob.glob(f"{dirs().input.sheets.outcomes}/*/第2から4層.csv"):
    l4=pd.read_csv(filename)
    l4=l4.rename(columns={"id":"y2022_id","H28対応項目":"y2016_id"})
    df= pd.concat([df,l4.loc[:,["y2016_id","y2022_id"]]])
df = expand_y2016_id(df)
save_csv(df,path.join(dirs().output.relations_2016,"y2016_layer5_to_y2022_layer4.csv"))
print(f"output... relations/y2016_layer5_to_y2022_layer4.csv")

# output y2016_layer5_to_y2022_tables.csv
df = pd.DataFrame([])
for table,info in iter_tables_for_outcome_raw():
    table = table\
        .rename(columns={"H28対応項目":"y2016_id","id":"y2022_id"})\
        .loc[:,["y2016_id","y2022_id"]]
    df= pd.concat([df,table])
df = expand_y2016_id(df)
save_csv(df,path.join(dirs().output.relations_2016,"y2016_layer5_to_y2022_tables.csv"))
print(f"output... relations/y2016_layer5_to_y2022_tables.csv")


In [ ]:
"""
output relations/y2016_layer5_to_y2022_others.csv
"""

import pandas as pd
import glob
import re
import csv
import os
from os import path
from lib.vars import dirs
from lib.utils import get_glob_file,save_csv,load_csv
from lib.tables_utils import iter_tables_for_outcome_raw

os.makedirs(f"{dirs().output.relations_2016}", exist_ok=True)
df = load_csv(path.join(dirs().input.sheets.general,"H28との対応(特殊な対応項目のみ)","H28との対応追加項目.csv"))\
    .rename(columns={"対応項目":"y2022_comment","H28ID":"y2016_id","id":"y2022_id"})
df = df.loc[:,["y2016_id","y2022_id","y2022_comment"]]
save_csv(df,path.join(dirs().output.relations_2016,"y2016_layer5_to_y2022_others.csv"))
print(f"output... relations/y2016_layer5_to_y2022_others.csv")
